In [2]:
import subprocess

In [3]:
print(subprocess.Popen("echo Hello World", shell=True, stdout=subprocess.PIPE).stdout.read())

b'Hello World\n'


In [33]:
jdk_path = subprocess.Popen("echo %JAVA_HOME%", shell=True, stdout=subprocess.PIPE).stdout.readline()    # to make sure jdk path is set
print(jdk_path)

b'%JAVA_HOME%\n'


In [143]:
os="LINUX" # LIINUX or WINDOWS

In [145]:
# don't change these unless you changed the location of these folders
if(os == "LINUX"):
    weka_path="./weka-3-4/weka.jar"
    COMMONS_CLI="./lib/commons-cli-1.0.jar" 
    JMRC="./lib/jmrc.jar"
    LIBS=".:"+weka_path+":"+COMMONS_CLI+":"+JMRC+":"+'bin/'     #+appDir+";"
    LINE_ENDING="\n"
    LINE_END_CHAR_COUNT=1
elif(os== "WINDOWS"):
    weka_path="weka-3-4\weka.jar"
    COMMONS_CLI="lib\commons-cli-1.0.jar" 
    JMRC="lib\jmrc.jar"
    LIBS=weka_path+";"+COMMONS_CLI+";"+JMRC+";"+'bin\\'     #+appDir+";"
    LINE_ENDING="\r\n"
    LINE_END_CHAR_COUNT=2

In [146]:
LIBS

'.:./weka-3-4/weka.jar:./lib/commons-cli-1.0.jar:./lib/jmrc.jar:bin/'

In [147]:
# in windows
#appDir = "E:\CiperLabs\\fiver\scripts\personality recognizer\PersonalityRecognizer"     #Directory where the java application is located
# in linux
appDir = "/media/gayan/Education/CiperLabs/fiver/scripts/personality recognizer/PersonalityRecognizer"

In [148]:
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [149]:
#Change the properties here to change the model parameters and files


# -d,--directory		Corpus analysis mode. Input must be a directory with 
#                          multiple text files, features are standardized over 
#                          the corpus and the recognizer outputs a personality 
#                          estimate for each text file.

input_type_is_dir = "-d"  #("set "-d" if folder or "" if file")

# -i,--input       	Input file or directory (required)
input_file_folder = "examples" 

#-m,--model       	Model to use for computing scores (default 4). Options:
#  	              				1 = Linear Regression
#                				2 = M5' Model Tree
#              					3 = M5' Regression Tree
#              					4 = Support Vector Machine with Linear Kernel (SMOreg)

model = "3" 

# -t,--type		Selects the type of model to use (default 1). The appropriate
#                         		model depends on the language sample (written or 
#   					spoken), and whether observed personality (as perceived 
#   					by external judges) or self-assessed personality (the 
#   					writer/speaker's perception) needs to be estimated from the 
#   					text. Options:
#   						1 = Observed personality from spoken language
#                                 		2 = Self-assessed personality from written language
model_type = "2"

In [150]:
def execPR():
    output = []
    # enter the app directory
    with cd(appDir):
       # we are in app directory when exiting the block we exit from the directory
        command = 'java -Xmx512m -classpath '+ LIBS+ ' recognizer.PersonalityRecognizer '+ input_type_is_dir + ' -i '+input_file_folder + ' -m '+model+' -t '+model_type
        p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        print(command)
        for line in p.stdout.readlines():
            output.append(line.decode("utf-8"))
        retval = p.wait()
    return output

In [151]:
outputstream = execPR()

java -Xmx512m -classpath .:./weka-3-4/weka.jar:./lib/commons-cli-1.0.jar:./lib/jmrc.jar:bin/ recognizer.PersonalityRecognizer -d -i examples -m 3 -t 2


In [153]:
output=[]                 #{filename,textprop,LIWC,MRC,extra,emoti,agree,consc,openn}
results = []
outStarted = False
for line in outputstream:
    if(line.startswith("Computing features for file")):
        name = line[28:][:-(3+LINE_END_CHAR_COUNT)]
        out = {}
        out["name"] =name
        index_of_arr = outputstream.index(line)
        out["textprop"] = outputstream[index_of_arr+1][:-LINE_END_CHAR_COUNT]
        out["LIWC"] =outputstream[index_of_arr+2][:-LINE_END_CHAR_COUNT]
        out["MRC"] = outputstream[index_of_arr+3][:-LINE_END_CHAR_COUNT]
        output.append(out)
        print(name)
    elif(line.startswith("File              \tExtra\tEmoti\tAgree\tConsc\tOpenn"+LINE_ENDING)):
        outStarted = True
    elif(line.startswith(LINE_ENDING)):
        outStarted = False
    elif(outStarted):
        results.append(line[:-LINE_END_CHAR_COUNT])

Amy.txt
John.txt
Matt.txt
Rob.txt


In [155]:
for i in range(len(output)):
    res = results[i].split(	)                # Splitting by tab
    output[i]["extra"] = res[1]
    output[i]["emoti"] = res[2]
    output[i]["agree"] = res[3]
    output[i]["consc"] = res[4]
    output[i]["openn"] = res[5]

In [168]:
import csv
toCSV = output
keys = toCSV[0].keys()
with open('output.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)